This notebook includes two JS code chunks to get the S tokens for simbli.

The code should be run on the console at https://sites.eboardsolutions.com/

The results are saved as `s_tokens.json`.

In [ ]:
// Select the dropdown element by its ID
const dropdown = document.querySelector('#ddlState');

// Extract the abbreviations from the <option> elements
const abbreviations = Array.from(dropdown.options)
    .map(option => option.value) // Get the value of each <option>
    .filter(value => value !== ""); // Exclude the empty value

// Log the abbreviations
console.log("Abbreviations:", abbreviations);

In [ ]:
(async function() {
    const url = "https://sites.eboardsolutions.com/Home/GetOrgByState";

    // List of state abbreviations
    const abbrList = abbreviations; // Add more abbreviations as needed

    // Retrieve the CSRF token from the page
    const csrfToken = document.querySelector('input[name="__RequestVerificationToken"]')?.value;
    if (!csrfToken) {
        console.error("CSRF token not found on the page!");
        return;
    }

    // Dictionary to store the results
    const resultsDict = {};

    // Function to send the request for a single abbreviation
    async function fetchData(abbr) {
        const payload = {
            abbr: abbr,
            __RequestVerificationToken: csrfToken
        };

        try {
            // Make the AJAX request
            const response = await fetch(url, {
                method: "POST",
                headers: {
                    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
                    "Accept": "application/json, text/javascript, */*; q=0.01",
                    "X-Requested-With": "XMLHttpRequest", // Indicates this is an AJAX request
                    "Referer": window.location.href // Uses the current page's URL as the Referer
                },
                body: new URLSearchParams(payload).toString() // Encodes payload as application/x-www-form-urlencoded
            });

            // Check response status
            if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);

            // Parse the response
            const data = await response.json();

            // Process the array under the key "d"
            if (data.d && Array.isArray(data.d)) {
                data.d.forEach(item => {
                    const { Id, Name, StateAbbr } = item;
                    if (Id && Name && StateAbbr) {
                        // Add to resultsDict
                        resultsDict[Id] = [Name, StateAbbr];
                    }
                });
            }
        } catch (error) {
            console.error(`Error fetching data for ${abbr}:`, error);
        }
    }

    // Loop through the list of abbreviations and fetch data for each
    for (const abbr of abbrList) {
        await fetchData(abbr);
    }

    // Log the final dictionary
    console.log("Results Dictionary:", resultsDict);

    // Save the resultsDict to a JSON file
    const saveAsFile = (data, filename) => {
        const blob = new Blob([JSON.stringify(data, null, 2)], { type: "application/json" });
        const url = URL.createObjectURL(blob);
        const a = document.createElement("a");
        a.href = url;
        a.download = filename;
        a.click();
        URL.revokeObjectURL(url); // Clean up the URL object
    };

    // Call the save function
    saveAsFile(resultsDict, "results.json");
})();